# Algorisme Estratègies Evolutives $(\mu+\lambda)$ amb auto-adaptació.



- Crear població inicial $\{x^1,...,x^\mu\}$. 
  Cada individu $x^i$ és de la forma $x^i=(x^i_1,..., x^i_n, \sigma_1, ..., \sigma_n)$, on $n$ és el nombre de paràmetres que voleu estimar.
- Avaluar cada membre de la població.
- Fins que es compleixi la condició de finalització:

  - Fins que es creïn $\lambda$ fills:
    - Seleccionar pare aleatòriament (un pare pot ser seleccionat múltiples vegades), i crear fill aplicant mutació. Avaluar fill. 
  - Ordenar els $\mu+\lambda$ individus segons fitness.
  - Seleccionar els millors $\mu$ per la següent generació.

*Mutació ("self-adaptive, n-step size"):*

Apliquem mutació gaussiana, on cada paràmetre té la seva desviació estàndard que s'autoadapta durant l'evolució. Per cada individu, la mutació de cada paràmetre $i, i = 1,...n$,  consisteix en dos passos: mutació de $\sigma_i$ i mutació del paràmetre $x_i$ segons la nova $\sigma_i$. 

- $\sigma_i(t+1)=\sigma_i(t) exp(\tau' r' + \tau \hat{r_i})$

- $x_i(t+1) = x_i(t) + \sigma_i(t+1)r_i$

    
On:
- $\tau' \propto 1 /\sqrt{2 n} $
- $\tau \propto 1/\sqrt{2 \sqrt n}$
- $r', \hat{r_1},...\hat{r_n}, r_1,...,r_n$ són i.i.d variables aleatòries extretes de la distribució normal estàndard $\mathcal{N}(0,1)$.








In [11]:
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from IPython import display
import copy
import math
import random

Classe individu

In [4]:
class individual:
   
    def __init__ (self, n=1, bslow = [0,0], bhigh = [1,1], g= [], fitness = 0, sigma = []):
       
        #definim els atributs
        self.ngenes = n
        self.fitness = fitness
        
       
        #rang del genotip
        self.boundslow = bslow
        self.boundhigh = bhigh
        
        
        if len(g) == 0:
            self.genotype = self.set_random_genotype()
            
        else: 
            self.genotype = np.array(g)
            
        if len(sigma)==0:
            self.sigma = self.set_sigma()
            
       
           
    #funció en cas que el genotip no estigui definit, tindrà un valor continu en el rang definit (boundslow i boundhigh)
    def set_random_genotype (self):
        self.genotype = np.random.uniform(low=self.boundslow, high=self.boundhigh, size = self.ngenes)
        
    #definim sigmes per cada numero de gens  
    def set_sigma(self):
        self.sigma = []
        for i in range(self.ngenes):
            self.sigma.append(0.5) #les definim totes iguals en un inici, despres s'aniran mutant 
     
        return self.sigma
    

Càlcul de la fitness

In [5]:

#funcio de fitness, calcul de la fitness de cada individu
def assign_fitness(x,y,individu):
    a = individu.genotype[0]
    b = individu.genotype[1]
    x = np.arange(1, 11)
        
    y = a * x + b
    ysoroll = y + np.random.normal(loc=0, scale=1, size=(10)) 
    
    MSE = np.mean(np.square(y - ysoroll))
    fitness = 1.0 / MSE
    
    return fitness


*Mutació ("self-adaptive, n-step size"):*

In [6]:
#funcio de mutacio que et retorna el fill (individu): copia del pare amb una x mutacio 
def mutate(individual):
    
    n = len(individual.genotype) 
    
    
    #fem una copia del pare per conservar el pare al crear el fill 
    fill = copy.deepcopy(individu) 
    
  
    
    tau_prime = 1 / (math.sqrt(2 * n))        
    tau = 1 / (math.sqrt(2 * math.sqrt(n)))


    for i in range(n):
        
        r_prime = np.random.normal(0,1)  # r prima    
        r_hat = np.random.normal(0,1)  # r i gorro
        r = np.random.normal(0,1)  # r i
        
        fill.sigma[i] = fill.sigma[i]*math.exp(tau_prime*r_prime + tau*r_hat)
    
        fill.genotype[i] = fill.genotype[i] + fill.sigma[i]*r  #apliquem la mutacio en el gen amb la nova tau
            

    return fill

Funcio de ordre i seleccio

In [7]:

#funcio que ens ordena els individus de millor a pitjor fitness
def ordre(poblacio):
    return list(sorted(poblacio, key=lambda atr: atr.fitness))



In [8]:
#funcio que et selecciona els individus amb millor fitness segons la mu que posem 

def select (poblacio, mu):
    
    if mu>= len(poblacio):
        
        return (poblacio)
        
    else: 
        
        return (poblacio[-mu:])

ALGORISME

In [9]:
def algorisme (poblacio, lamb, mu, x, y):
    
    #calculem fitness dels pares i dels fills 

    for individu in poblacio:
        assign_fitness (x,y,individu)
    
    #bucle que aniran entran cada generacio 
    for i in range (100):
    
        #creem la poblacio dels fills mutats 
        poblacio_fills = []

        for i in range (lamb): #generem lamb fills
            individu_pare = random.randint(0,len(poblacio)-1) #seleccionem un pare
            fills = mutate(poblacio[individu_pare]) #creem el fill a partir d'aquell pare + mutacio
            poblacio_fills.append(fills) #afegim els fills a una llista
            assign_fitness (x,y,fills) #calculem la fitness dels fills
            

        #afegim els fills a la poblacio
        poblacio.extend(poblacio_fills)

        #ordenem tota la poblacio segons la fitness 
        poblacio_ordenada = ordre(poblacio)

        #seleccionem els individus amb millor fitness per la seguent generacio
        seleccio_poblacio = select(poblacio_ordenada, mu)
    
        #actualizem la poblacio, que torna a entrar al bucle 100 vegades
        poblacio = seleccio_poblacio
    
    bestindividu = poblacio[0]
    
    return  bestindividu 
    

Apartir d'un dataset, escollim el millor individu despres de x generacions

In [12]:
x = np.arange(10)
y = x + 300 #creem la recta que volem aproximar
poblacio = []

#ho comprovem generant una poblacio de 10 individus 
for i in range(1000): 
    g1 = random.randint(1, 10)
    g2 = random.randint(1, 10)
    individu = individual(n=2, g=[g1, g2])
    poblacio.append(individu) # Añadimos a la población los individuos que vamos creando

bestindividual = algorisme(poblacio, 80, 120, x, y) 

print(bestindividual.genotype)

[0 3]
